In [1]:
import string

# Specify the path to the text file
file_path = 'data\gutmgene\smiles_ids.txt'
# Initialize an empty list to store the SMILES strings
smiles_dict = {}
# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        smiles, id = line.split(" ")
        smiles = smiles.strip()
        id = id.strip()
        smiles_dict[id] = smiles

print(len(smiles_dict))

# Define what counts as an atom (letters only)
def count_atoms(smiles):
    return sum(1 for c in smiles if c.isalpha())

# Filter the dictionary
smiles_dict = {id: smi for id, smi in smiles_dict.items() if count_atoms(smi) >= 5}
smiles_dict = {key: value.replace('.', '') for key, value in smiles_dict.items()}

print(len(smiles_dict))

246
236


<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\vitor\AppData\Local\Temp\ipykernel_3900\1916485544.py:4: SyntaxWarning: invalid escape sequence '\g'
  file_path = 'data\gutmgene\smiles_ids.txt'


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoAlertPresentException
from selenium.common.exceptions import WebDriverException
import pathlib as p
import csv
import time
import shutil
import os
import random

#chrome_path = p.Path(r"C:\Users\usuario\Documents\vitoria\usp-masters\data\stp\chromedriver-win64\\chromedriver.exe").absolute()
#user_data = p.Path(r"C:\\Users\\usuario\\Documents\\vitoria\\usp-masters\\data\\stp\\server").absolute()
#download_path = p.Path(r"C:\\Users\\usuario\\Documents\\vitoria\\usp-masters\\data\\stp").absolute()

#chrome_path = p.Path(r"C:\\Users\\jvsam\\OneDrive\\Documentos\\masters\\usp-masters\\data\\stp\\chromedriver-win64\\chromedriver.exe").absolute()
#user_data = p.Path(r"C:\\Users\\jvsam\\OneDrive\\Documentos\\masters\\usp-masters\\data\\stp\\server").absolute()
#download_path = p.Path(r"C:\\Users\\jvsam\\OneDrive\\Documentos\\masters\\usp-masters\\data\\stp").absolute()

chrome_path = p.Path(r"C:\\Users\\vitor\\Documents\\usp\\usp-masters\\data\\stp\\chromedriver-win64\\chromedriver.exe").absolute()
user_data = p.Path(r"C:\\Users\\vitor\\Documents\\usp\\usp-masters\\data\\stp\\server").absolute()
download_path = p.Path(r"C:\\Users\\vitor\\Documents\\usp\\usp-masters\\data\\stp").absolute()

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=' + str(user_data))

service = Service(chrome_path)
prefs = {
    "download.default_directory": str(download_path),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("prefs", prefs)

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15"
]
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

driver = webdriver.Chrome(service=service, options=options)  

def is_browser_open(driver):
    try:
        # A simple command to test if browser is still reachable
        driver.title
        return True
    except WebDriverException:
        return False
    
def human_delay(min_sec=10, max_sec=20):
    time.sleep(random.uniform(min_sec, max_sec))

def predict_targets_stf(smiles, output_file, retries=3):
    for attempt in range(retries):
        try:
            driver.get('http://www.swisstargetprediction.ch/')
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#smilesBox'))).click()
            driver.find_element(By.CSS_SELECTOR, '#smilesBox').send_keys(smiles)    
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#submitButton'))).click()
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "button.dt-button.buttons-csv"))
            ).click()

            timeout = 30
            downloaded_file = None
            for _ in range(timeout):
                for filename in os.listdir(download_path):
                    if filename.endswith("SwissTargetPrediction.csv"):
                        downloaded_file = os.path.join(download_path, filename)
                        break
                if downloaded_file:
                    break
                time.sleep(1)

            if downloaded_file:
                final_path = os.path.join(download_path, output_file)
                shutil.move(downloaded_file, final_path)
                return
            else:
                raise Exception("Download failed or timed out.")
        except Exception as e:
            # print(f"Attempt {attempt + 1} failed: {e}")
            human_delay(10, 30)
    # print(f"Failed to process SMILES: {smiles}")

for idx, (id, smiles) in enumerate(filtered_dict.items()):
    if not is_browser_open(driver):
        print("Browser was closed. Exiting script.")
        break
    
    print(f"Processing {id} ({idx + 1}/{len(smiles_dict)})...")
    output_file = f"targets_{id}.csv"
    print(smiles)
    predict_targets_stf(smiles, output_file)
    human_delay(15, 30)

    # Restart browser every N iterations
    if (idx + 1) % 10 == 0:
        driver.quit()
        time.sleep(60)  # Cooldown period
        driver = webdriver.Chrome(service=service, options=options)

driver.quit()

Processing cid442424 (1/236)...
COC(=O)C1=COC(C2C1CC=C2CO)O
Attempt 1 failed: Alert Text: No similar actives found. No target predicted.
Message: unexpected alert open: {Alert text : No similar actives found. No target predicted.}
  (Session info: chrome=135.0.7049.85)
Stacktrace:
	GetHandleVerifier [0x00007FF6577C1F55+78133]
	GetHandleVerifier [0x00007FF6577C1FB0+78224]
	(No symbol) [0x00007FF6575891BA]
	(No symbol) [0x00007FF657599C29]
	(No symbol) [0x00007FF6575A08BA]
	(No symbol) [0x00007FF65757509D]
	(No symbol) [0x00007FF657599101]
	(No symbol) [0x00007FF65762F0F9]
	(No symbol) [0x00007FF657606F33]
	(No symbol) [0x00007FF6575D0358]
	(No symbol) [0x00007FF6575D10C3]
	GetHandleVerifier [0x00007FF657A8BA8D+3001453]
	GetHandleVerifier [0x00007FF657A85E72+2977874]
	GetHandleVerifier [0x00007FF657AA497D+3103581]
	GetHandleVerifier [0x00007FF6577DC7EA+186826]
	GetHandleVerifier [0x00007FF6577E43FF+218591]
	GetHandleVerifier [0x00007FF6577C9D94+110452]
	GetHandleVerifier [0x00007FF6577C9

KeyboardInterrupt: 

In [ ]:
import os

# Caminhos dos arquivos e diretórios
smiles_ids_file = "data/smiles_ids.txt"
csv_dir = "data/target_prediction/stp/"

# Lendo os IDs do arquivo smiles_ids.txt
with open(smiles_ids_file, "r") as file:
    ids = [line.strip().split()[1] for line in file]

# Verificando quais IDs não têm arquivos CSV
missing_ids = [id_ for id_ in ids if not os.path.exists(os.path.join(csv_dir, f"{id_}.csv"))]

# Imprimindo os IDs ausentes
print("IDs sem arquivo CSV:")
print("\n".join(missing_ids))

# Criando os arquivos CSV para os IDs ausentes
for id_ in missing_ids:
    with open(os.path.join(csv_dir, f"{id_}.csv"), "w") as file:
        file.write("No targets predicted.\n")

print("Arquivos CSV criados para os IDs ausentes.")


IDs sem arquivo CSV:
cid1146
cid175
cid702
cid283
cid223
cid280
cid1031
cid783
nocid1
nocid2
nocid3
nocid4
nocid5
nocid12
nocid13
nocid14
nocid15
nocid32
nocid33
nocid34
Arquivos CSV criados para os IDs ausentes.


In [7]:
import time
import os
import shutil

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
    
URL = 'https://chembl.github.io/chembl_multitask_model/'

# Inicializa o driver (usando Chrome)
DOWNLOAD_DIR = os.path.abspath(r"C:\Users\vitor\Downloads\chemblpred")  # or your chosen folder
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
driver = webdriver.Chrome(options=options)


def wait_and_rename(download_dir, new_name, timeout=30):
    """Waits for predictions.csv to appear and renames it."""
    target_file = os.path.join(download_dir, "predictions.csv")
    start = time.time()
    while time.time() - start < timeout:
        if os.path.exists(target_file):
            # Rename when found
            new_path = os.path.join(download_dir, f"{new_name}.csv")
            shutil.move(target_file, new_path)
            print(f"Saved as {new_name}.csv")
            return
        time.sleep(1)
    raise TimeoutError(f"Download for {new_name} not found in time.")

for id, smiles in smiles_dict.items():   

    try:
        driver.get(URL)

        input_box = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.ID, 'input-3')))

        time.sleep(5)

        input_box.clear()
        input_box.send_keys(smiles)

        WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, "//th//span[text()='Probability']"))
        )

        time.sleep(2) 

        # Once table is visible, click Download Results
        download_button = driver.find_element(By.XPATH, "//span[contains(text(), 'Download Results')]/..")
        driver.execute_script("arguments[0].scrollIntoView(true);", download_button)
        download_button.click()
        # driver.execute_script("arguments[0].click();", download_button)

        wait_and_rename(DOWNLOAD_DIR, id)

        time.sleep(5)

    except Exception as e:
        print(f"Method 1 failed: {e}")
        
        # Method 2: Try finding by button tag
        try:
            download_button = driver.find_element(By.XPATH, "//button[contains(., 'Download Results')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", download_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", download_button)
            print("Clicked using Method 2")

            wait_and_rename(DOWNLOAD_DIR, id)
        except Exception as e2:
            print(f"Method 2 also failed: {e2}")


driver.quit()


Method 1 failed: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[contains(text(), 'Download Results')]/.."}
  (Session info: chrome=140.0.7339.208); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff636531eb5+80197]
	GetHandleVerifier [0x0x7ff636531f10+80288]
	(No symbol) [0x0x7ff6362b02fa]
	(No symbol) [0x0x7ff636307cd7]
	(No symbol) [0x0x7ff636307f9c]
	(No symbol) [0x0x7ff63635ba87]
	(No symbol) [0x0x7ff6363303bf]
	(No symbol) [0x0x7ff6363587fb]
	(No symbol) [0x0x7ff636330153]
	(No symbol) [0x0x7ff6362f8b02]
	(No symbol) [0x0x7ff6362f98d3]
	GetHandleVerifier [0x0x7ff6367ee83d+2949837]
	GetHandleVerifier [0x0x7ff6367e8c6a+2926330]
	GetHandleVerifier [0x0x7ff6368086c7+3055959]
	GetHandleVerifier [0x0x7ff63654cfee+191102]
	GetHandleVerifier [0x0x7ff6365550af+224063]
	GetHandleVerifier [0x0x7ff63653af64+117236]
	GetHan

In [ ]:
import time
import os
import shutil

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


URL = 'https://ppb2.gdb.tools/?ref=gdb.unibe.ch#'

# Inicializa o driver (usando Chrome)
DOWNLOAD_DIR = os.path.abspath(r"C:\Users\vitor\Downloads\ppb2")  # or your chosen folder
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.set_page_load_timeout(600)
driver.set_script_timeout(600)


wait = WebDriverWait(driver, 60)  # waits up to 10 seconds

def wait_and_rename(download_dir, new_name, timeout=30):
    """Waits for predictions.csv to appear and renames it."""
    target_file = os.path.join(download_dir, "predictionResult.txt")
    start = time.time()
    while time.time() - start < timeout:
        if os.path.exists(target_file):
            # Rename when found
            new_path = os.path.join(download_dir, f"{new_name}.txt")
            shutil.move(target_file, new_path)
            # print(f"Saved as {new_name}.csv")
            return
        time.sleep(1)
    raise TimeoutError(f"Download for {new_name} not found in time.")

def safe_click(driver, el):
    # scroll para centro
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.2)
    # tentativa com ActionChains (mais "humana")
    try:
        ActionChains(driver).move_to_element(el).click().perform()
        return
    except Exception:
        pass
    # fallback: click() via JS
    try:
        driver.execute_script("arguments[0].click();", el)
        return
    except Exception:
        pass
    # último recurso: dispatch de MouseEvent (bom para shadow DOM / handlers)
    driver.execute_script("""
        var el = arguments[0];
        var ev = new MouseEvent('click', {
            view: window, bubbles: true, cancelable: true, composed: true
        });
        el.dispatchEvent(ev);
    """, el)

true_original_handle = driver.current_window_handle

i = 35

for id, smiles in list(smiles_dict.items())[35:]:   
    try:
        driver.get(URL)
        time.sleep(3) 
        driver.refresh()
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.jsa-resetDiv"))
        )

        # Locate the canvas or button you want to right-click
        drawing_box = driver.find_element(By.CSS_SELECTOR, "div.jsa-resetDiv")

        # Perform right-click
        actions = ActionChains(driver)
        actions.move_to_element_with_offset(drawing_box, 5, 5).context_click().perform()

        # Now select the menu item that appeared
        menu_item = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//td[contains(text(), 'Paste MOL or SDF or SMILES')]"))
        )
        menu_item.click()

        # Now the paste modal appears, you can send your SMILES string
        paste_box = WebDriverWait(driver, 5).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "textarea.gwt-TextArea"))
        )
        paste_box.send_keys(smiles)

        # Confirm
        accept_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Accept']"))
        )
        accept_button.click()
        time.sleep(2) 

        #model_ids = ["NNML_ECfp4", "NNML_Xfp"]
        model_ids = ["NN_ECfp4", "NN_Xfp", "NN_MQN", "NNML_MQN"]

        # Store the handle of the page where we just pasted the SMILES
        smiles_page_handle = driver.current_window_handle
        
        for mid in model_ids:
            # print(f"Processing SMILES {id}, model: {mid}")

            # Always go back to the SMILES page before clicking a new model
            driver.switch_to.window(smiles_page_handle)
            time.sleep(1)

            # Find and click the model link
            model_link = wait.until(EC.presence_of_element_located((By.ID, mid)))
            
            #print(f"DEBUG: {model_link.tag_name}, href={model_link.get_attribute('href')}, "
                  #f"onclick={model_link.get_attribute('onclick')}, "
                  #f"displayed={model_link.is_displayed()}, enabled={model_link.is_enabled()}")

            # Click to open in new tab
            before_handles = set(driver.window_handles)
            safe_click(driver, model_link)

            # Wait for new tab to appear
            WebDriverWait(driver, 10).until(lambda d: len(set(d.window_handles) - before_handles) > 0)
            
            # Find the new tab handle
            new_handles = set(driver.window_handles) - before_handles
            if not new_handles:
                # print(f"ERROR: No new tab opened for {mid}")
                continue
                
            results_tab_handle = new_handles.pop()
            driver.switch_to.window(results_tab_handle)
            # print(f"Switched to results tab for {mid}")
            time.sleep(5)

            # Wait for and click the save button
            try:
                save_button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.btn-danger[onclick*='saveTable']"))
                )
                save_button.click()
                # print("Clicked Save Table")
                time.sleep(2)

                wait_and_rename(DOWNLOAD_DIR, f"{id}_{mid}")
            except Exception as e:
                print(f"ERROR downloading results for {mid}: {e}")

            # Close ONLY the results tab
            driver.close()
            # print(f"Closed results tab for {mid}")
            
            # Switch back to the SMILES page for the next model
            driver.switch_to.window(smiles_page_handle)
            time.sleep(1)

        # After processing both models, verify we're on the right tab
        driver.switch_to.window(true_original_handle)
        
        # Close any extra tabs that shouldn't be there (cleanup)
        all_handles = driver.window_handles
        for handle in all_handles:
            if handle != true_original_handle:
                driver.switch_to.window(handle)
                driver.close()
        
        driver.switch_to.window(true_original_handle)
        print(f"Completed SMILES {i} - {id}, cleaned up tabs")
        i = i + 1
        time.sleep(2)

    except Exception as e:
        # print(f"Failed processing SMILES {id}: {e}")
        
        # Emergency cleanup: close all tabs except original
        try:
            for handle in driver.window_handles:
                if handle != true_original_handle:
                    driver.switch_to.window(handle)
                    driver.close()
            driver.switch_to.window(true_original_handle)
        except:
            pass

driver.quit()

In [ ]:
list(smiles_dict.items())[35]

('cid91435', 'CC(C(=O)[O-])O')

In [47]:
import os
from collections import defaultdict

# Your list of expected IDs
expected_ids = list(smiles_dict.keys())  # replace with your actual list

# Directory containing the .txt files
directory = os.path.abspath(r"C:\Users\vitor\Downloads\ppb2")  # or your chosen folder


# Dictionary to count files per ID
id_counts = defaultdict(int)

# Scan the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Extract ID from filename (assuming ID is part of the filename)
        for eid in expected_ids:
            if eid in filename:
                id_counts[eid] += 1

# Find missing and incomplete IDs
missing_ids = [eid for eid in expected_ids if eid not in id_counts]
incomplete_ids = [eid for eid, count in id_counts.items() if count == 1]

print("Missing IDs:", missing_ids)
print("IDs with only one file:", incomplete_ids)


Missing IDs: []
IDs with only one file: []
